# point prediction to uncertainty

Xiao Song

<https://xsong.ltd/en>     
[Kaggle profile](https://www.kaggle.com/rikdifos/)

The competition webpage: <https://www.kaggle.com/c/m5-forecasting-uncertainty>

+ This notebook is a [public kernel](https://www.kaggle.com/szmnkrisz97/point-to-uncertainty-different-ranges-per-level) released by [KrisztianSz](https://www.kaggle.com/szmnkrisz97). It convert accuracy prediction to uncertainty.  

+ Input: 'submission_accuracy_averaging.csv'.   
+ Output: 'from_accuracy2uncertainty.csv'

> My aim is to show that uncertainty, in some sense, decreases on higher levels of aggregation.  
The bulk of the work in this kernel is done by [@kneroma](https://www.kaggle.com/kneroma) in [this kernel](https://www.kaggle.com/kneroma/from-point-to-uncertainty-prediction), so make sure to check it out.



In [1]:
import pandas as pd, numpy as np
from matplotlib import pyplot as plt
import scipy.stats  as stats

In [2]:
pd.options.display.max_columns = 50

In [3]:
best = pd.read_csv("../input/submission_from_accuracy_averaged.csv")
best.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,id
0,0.851810,0.784350,0.756261,0.757347,0.788509,0.956125,0.930274,0.740840,0.825155,0.870190,0.866647,1.040569,1.192166,1.141793,0.890297,0.791801,0.776584,0.782809,0.848619,1.141859,1.180419,0.886943,0.829009,0.763599,0.769547,0.792146,0.883108,0.834700,FOODS_1_001_CA_1_evaluation
1,0.851810,0.784350,0.756261,0.757347,0.788509,0.956125,0.930274,0.740840,0.825155,0.870190,0.866647,1.040569,1.192166,1.141793,0.890297,0.791801,0.776584,0.782809,0.848619,1.141859,1.180419,0.886943,0.829009,0.763599,0.769547,0.792146,0.883108,0.834700,FOODS_1_001_CA_1_validation
2,1.005145,1.055328,1.021212,0.945167,1.052385,1.343989,1.372671,0.950686,0.861435,0.910386,0.907855,0.951031,1.105475,1.056918,0.821309,0.824996,0.827798,0.834469,0.925459,1.122967,1.068700,0.817361,0.732840,0.723035,0.706774,0.769960,0.945644,0.886650,FOODS_1_001_CA_2_evaluation
3,1.005145,1.055328,1.021212,0.945167,1.052385,1.343989,1.372671,0.950686,0.861435,0.910386,0.907855,0.951031,1.105475,1.056918,0.821309,0.824996,0.827798,0.834469,0.925459,1.122967,1.068700,0.817361,0.732840,0.723035,0.706774,0.769960,0.945644,0.886650,FOODS_1_001_CA_2_validation
4,0.886574,0.856182,0.882874,0.876274,1.199584,1.421223,1.433368,1.084766,1.053409,1.143178,1.156499,1.105912,1.265417,1.230287,1.026057,0.975068,0.977930,0.989962,1.029569,1.118539,1.135825,0.969928,0.883021,0.876764,0.863847,0.970628,1.167420,1.153912,FOODS_1_001_CA_3_evaluation


In [4]:
sales = pd.read_csv("../data/sales_train_validation.csv")
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,...,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4


In [5]:
sub = best.merge(sales[["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]], on = "id")
sub["_all_"] = "Total"
sub.shape

(30490, 35)

In [6]:
sub.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,id,item_id,dept_id,cat_id,store_id,state_id,_all_
0,0.851810,0.784350,0.756261,0.757347,0.788509,0.956125,0.930274,0.740840,0.825155,0.870190,0.866647,1.040569,1.192166,1.141793,0.890297,0.791801,0.776584,0.782809,0.848619,1.141859,1.180419,0.886943,0.829009,0.763599,0.769547,0.792146,0.883108,0.834700,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,Total
1,1.005145,1.055328,1.021212,0.945167,1.052385,1.343989,1.372671,0.950686,0.861435,0.910386,0.907855,0.951031,1.105475,1.056918,0.821309,0.824996,0.827798,0.834469,0.925459,1.122967,1.068700,0.817361,0.732840,0.723035,0.706774,0.769960,0.945644,0.886650,FOODS_1_001_CA_2_validation,FOODS_1_001,FOODS_1,FOODS,CA_2,CA,Total
2,0.886574,0.856182,0.882874,0.876274,1.199584,1.421223,1.433368,1.084766,1.053409,1.143178,1.156499,1.105912,1.265417,1.230287,1.026057,0.975068,0.977930,0.989962,1.029569,1.118539,1.135825,0.969928,0.883021,0.876764,0.863847,0.970628,1.167420,1.153912,FOODS_1_001_CA_3_validation,FOODS_1_001,FOODS_1,FOODS,CA_3,CA,Total
3,0.311681,0.337344,0.342794,0.348719,0.340833,0.365301,0.377065,0.342905,0.302564,0.317469,0.314681,0.334197,0.372208,0.348226,0.302074,0.290465,0.287943,0.290843,0.320184,0.345301,0.350305,0.302284,0.292788,0.307418,0.326870,0.324495,0.361403,0.327455,FOODS_1_001_CA_4_validation,FOODS_1_001,FOODS_1,FOODS,CA_4,CA,Total
4,0.323149,0.413686,0.340635,0.377885,0.430933,0.449446,0.522656,0.465097,0.447663,0.501736,0.499975,0.518945,0.658975,0.668660,0.529923,0.525650,0.510742,0.535700,0.595875,0.692808,0.719990,0.607713,0.759066,0.754640,0.781162,0.886481,1.047443,1.049474,FOODS_1_001_TX_1_validation,FOODS_1_001,FOODS_1,FOODS,TX_1,TX,Total


## Different ratios for different aggregation levels

The higher the aggregation level, the more confident we are in the point prediction --> lower coef, relatively smaller range of quantiles

In [7]:
qs = np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995])
qs.shape


def get_ratios(coef=0.15):
    qs2 = np.log(qs/(1-qs))*coef
    ratios = stats.norm.cdf(qs2)
    ratios /= ratios[4]
    ratios = pd.Series(ratios, index=qs)
    return ratios.round(3)

#coef between 0.05 and 0.24 is used, probably suboptimal values for now

level_coef_dict = {"id": get_ratios(coef=0.3), "item_id": get_ratios(coef=0.15),
                   "dept_id": get_ratios(coef=0.08), "cat_id": get_ratios(coef=0.07),
                   "store_id": get_ratios(coef=0.08), "state_id": get_ratios(coef=0.07), "_all_": get_ratios(coef=0.05),
                   ("state_id", "item_id"): get_ratios(coef=0.19),  ("state_id", "dept_id"): get_ratios(coef=0.1),
                    ("store_id","dept_id") : get_ratios(coef=0.11), ("state_id", "cat_id"): get_ratios(coef=0.08),
                    ("store_id","cat_id"): get_ratios(coef=0.1)
                  }



Let's see how ranges differ!

In [8]:
level_coef_dict["id"]

0.005    0.112
0.025    0.272
0.165    0.627
0.250    0.742
0.500    1.000
0.750    1.258
0.835    1.373
0.975    1.728
0.995    1.888
dtype: float64

In [9]:
level_coef_dict["cat_id"]

0.005    0.711
0.025    0.798
0.165    0.910
0.250    0.939
0.500    1.000
0.750    1.061
0.835    1.090
0.975    1.202
0.995    1.289
dtype: float64

For the the lowest level (30490 series), the smallest and biggest quantiles are 20% and 180% of the point prediction. For categories (3 series), the model will be way more confident: the smallest quantile will be 71%, the biggest will be 129% of the point prediction.

In [10]:


def quantile_coefs(q, level):
    ratios = level_coef_dict[level]
               
    return ratios.loc[q].values

def get_group_preds(pred, level):
    df = pred.groupby(level)[cols].sum()
    q = np.repeat(qs, len(df))
    df = pd.concat([df]*9, axis=0, sort=False)
    df.reset_index(inplace = True)
    df[cols] *= quantile_coefs(q, level)[:, None]
    if level != "id":
        df["id"] = [f"{lev}_X_{q:.3f}_validation" for lev, q in zip(df[level].values, q)]
    else:
        df["id"] = [f"{lev.replace('_validation', '')}_{q:.3f}_validation" for lev, q in zip(df[level].values, q)]
    df = df[["id"]+list(cols)]
    return df

def get_couple_group_preds(pred, level1, level2):
    df = pred.groupby([level1, level2])[cols].sum()
    q = np.repeat(qs, len(df))
    df = pd.concat([df]*9, axis=0, sort=False)
    df.reset_index(inplace = True)
    df[cols] *= quantile_coefs(q, (level1, level2))[:, None]
    df["id"] = [f"{lev1}_{lev2}_{q:.3f}_validation" for lev1,lev2, q in 
                zip(df[level1].values,df[level2].values, q)]
    df = df[["id"]+list(cols)]
    return df

levels = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "_all_"]
couples = [("state_id", "item_id"),  ("state_id", "dept_id"),("store_id","dept_id"),
                            ("state_id", "cat_id"),("store_id","cat_id")]
cols = [f"F{i}" for i in range(1, 29)]

df = []
for level in levels :
    df.append(get_group_preds(sub, level))
for level1,level2 in couples:
    df.append(get_couple_group_preds(sub, level1, level2))
df = pd.concat(df, axis=0, sort=False)
df.reset_index(drop=True, inplace=True)
df = pd.concat([df,df] , axis=0, sort=False)
df.reset_index(drop=True, inplace=True)
df.loc[df.index >= len(df.index)//2, "id"] = df.loc[df.index >= len(df.index)//2, "id"].str.replace(
                                    "_validation", "_evaluation")



In [11]:
print(df.shape)

df.head()

(771120, 29)


,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_0.005_validation,0.095403,0.087847,0.084701,0.084823,0.088313,0.107086,0.104191,0.082974,0.092417,0.097461,0.097064,0.116544,0.133523,0.127881,0.099713,0.088682,0.086977,0.087675,0.095045,0.127888,0.132207,0.099338,0.092849,0.085523,0.086189,0.088720,0.098908,0.093486
1,FOODS_1_001_CA_2_0.005_validation,0.112576,0.118197,0.114376,0.105859,0.117867,0.150527,0.153739,0.106477,0.096481,0.101963,0.101680,0.106516,0.123813,0.118375,0.091987,0.092399,0.092713,0.093461,0.103651,0.125772,0.119694,0.091544,0.082078,0.080980,0.079159,0.086236,0.105912,0.099305
2,FOODS_1_001_CA_3_0.005_validation,0.099296,0.095892,0.098882,0.098143,0.134353,0.159177,0.160537,0.121494,0.117982,0.128036,0.129528,0.123862,0.141727,0.137792,0.114918,0.109208,0.109528,0.110876,0.115312,0.125276,0.127212,0.108632,0.098898,0.098198,0.096751,0.108710,0.130751,0.129238
3,FOODS_1_001_CA_4_0.005_validation,0.034908,0.037783,0.038393,0.039056,0.038173,0.040914,0.042231,0.038405,0.033887,0.035557,0.035244,0.037430,0.041687,0.039001,0.033832,0.032532,0.032250,0.032574,0.035861,0.038674,0.039234,0.033856,0.032792,0.034431,0.036609,0.036343,0.040477,0.036675
4,FOODS_1_001_TX_1_0.005_validation,0.036193,0.046333,0.038151,0.042323,0.048264,0.050338,0.058538,0.052091,0.050138,0.056194,0.055997,0.058122,0.073805,0.074890,0.059351,0.058873,0.057203,0.059998,0.066738,0.077595,0.080639,0.068064,0.085015,0.084520,0.087490,0.099286,0.117314,0.117541


In [12]:
df.to_csv("from_accuracy2uncertainty.csv", index = False)